In [1]:
%%capture
!pip install pycocotools -q
!pip install --upgrade wandb -q
!pip3 install pytorch-lightning transformers wandb -q


In [2]:
!pip3 install deepspeed -q

     |████████████████████████████████| 687 kB 27.3 MB/s 
     |████████████████████████████████| 54 kB 901 kB/s 
     |████████████████████████████████| 145 kB 27.8 MB/s 


In [3]:
!pip install torch torchvision torchaudio -q

In [4]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [5]:
from __future__ import print_function, division
import os
import torch
import string
from typing import Callable, Dict, List, Optional, Tuple, Union
from torch.utils.data import Dataset, DataLoader, random_split
import torchvision.datasets as dset
import random
from transformers import EncoderDecoderModel, GPT2Tokenizer, ViTFeatureExtractor, AutoTokenizer
from tqdm.auto import tqdm
from pytorch_lightning.callbacks import ModelCheckpoint
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import torchvision.datasets as dset
import multiprocessing as mp
from PIL import Image
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
import cv2
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from deepspeed.ops.adam import FusedAdam
from pytorch_lightning.loggers import WandbLogger
import wandb

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

plt.ion()   # interactive mode

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Config

In [7]:
VIT_MODEL = "google/vit-base-patch16-224-in21k"
GPT2 = "gpt2"
DISTIL_GPT2 = "distilgpt2"

# DATA_PATH = "/content/drive/MyDrive/CS640 Project/AI Project"

DATA_PATH = "/content/drive/MyDrive/AI Project/CS640 Project/Processed Frames/"
ANNOTATION_PATH = "/content/drive/MyDrive/AI Project/CS640 Project/Y.json"

MEAN = (0.485, 0.456, 0.406)
STD = (0.229, 0.224, 0.225)

TRAIN_PCT = 0.95
NUM_WORKERS = mp.cpu_count()
BATCH_SIZE = 8
EPOCHS = 10
LR = 1e-4
IMAGE_SIZE = (224, 224)

MAX_TEXT_LENGTH = 32

LABEL_MASK = -100

TOP_K = 1000
TOP_P = 0.95

In [9]:
# y = pd.read_csv('/content/drive/MyDrive/AI Project/CS640 Project/Y.csv')
# coco = pd.read_csv('/content/drive/MyDrive/AI Project/CS640 Project/COCO_labels.csv')
# y.to_json('/content/drive/MyDrive/CS640 Project/Y.json')

In [ ]:
# coco.rename(columns={'label': 'labels'}).to_csv('/content/drive/MyDrive/CS640 Project/COCO_labels.csv', index=False)

## Data

In [8]:
class GifDataset(Dataset):
    """Gif Caption dataset."""

    def __init__(self, csv_file, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.labels = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    
    def _load_image(self, id: int):
        img_name = os.path.join(self.root_dir, self.labels.iloc[id, 0])
        return Image.open(img_name)

    def _load_target(self, id: int):
      labels = self.labels.iloc[id, 0]
      return np.array([labels])
    
    # def __getitem__(self, index: int):
    #     id = index
    #     image = self._load_image(id)
    #     target = self._load_target(id)

        
        
    #     sample = {'image': image, 'labels': target}

    #     if self.transform is not None:
    #         image, target = self.transform(sample)

    #     return sample

    def __getitem__(self, idx):
        # print(idx)
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.root_dir,
                                self.labels.iloc[idx, 0])

        # image = io.imread(img_name)
        image = Image.open(img_name).convert('RGB')
        # image = np.array(image)
        
        labels = self.labels.iloc[idx, 0]
        # labels = np.array([labels])
        
        # labels = labels.astype('float').reshape(-1, 2)
        sample = {'image': image, 'labels': labels, 'idx': idx}
        
        if self.transform:
            # print(sample['labels'].shape)
            sample['image'] = self.transform(sample['image'])
            
        return (sample['image'], labels)

      

In [11]:
# class Rescale(object):
#     """Rescale the image in a sample to a given size.

#     Args:
#         output_size (tuple or int): Desired output size. If tuple, output is
#             matched to output_size. If int, smaller of image edges is matched
#             to output_size keeping aspect ratio the same.
#     """

#     def __init__(self, output_size):
#         assert isinstance(output_size, (int, tuple))
#         self.output_size = output_size

#     def __call__(self, sample):
#         image, labels = sample['image'], sample['labels']

#         h, w = image.shape[:2]
#         if isinstance(self.output_size, int):
#             if h > w:
#                 new_h, new_w = self.output_size * h / w, self.output_size
#             else:
#                 new_h, new_w = self.output_size, self.output_size * w / h
#         else:
#             new_h, new_w = self.output_size

#         new_h, new_w = int(new_h), int(new_w)

#         img = transform.resize(image, (new_h, new_w))

#         # h and w are swapped for landmarks because for images,
#         # x and y axes are axis 1 and 0 respectively
#         # landmarks = landmarks * [new_w / w, new_h / h]
#         # print(landmarks)

#         return {'image': img, 'labels': labels}


# class ToTensor(object):
#     """Convert ndarrays in sample to Tensors."""

#     def __call__(self, sample):
#         image, labels = sample['image'], sample['labels']

#         # swap color axis because
#         # numpy image: H x W x C
#         # torch image: C X H X W
#         image = image.transpose((2, 0, 1))
#         # print(landmarks)
#         # return {'image': torch.from_numpy(image),
#         #         'labels': landmarks}
#         # print(landmarks)
#         # return {'image': torch.from_numpy(image), 'labels': labels}
#         labels = labels.replace('.png','')
#         labels = labels.translate(str.maketrans('', '', string.punctuation))
#         return (torch.from_numpy(image).float(), labels)

# def show_gifs(image, labels):
#     """Show image with landmarks"""
#     plt.imshow(image)
#     plt.scatter(labels[:, 0], labels[:, 1], s=10, marker='.', c='r')
#     plt.pause(0.001)


# scale = Rescale(512)
# composed = transforms.Compose([Rescale((224,224))])

In [12]:
# class TestLoader(Dataset):
#     """Gif Caption dataset."""

#     def __init__(self, root_dir, transform=None):
#         """
#         Args:
#             csv_file (string): Path to the csv file with annotations.
#             root_dir (string): Directory with all the images.
#             transform (callable, optional): Optional transform to be applied
#                 on a sample.
#         """
#         self.root_dir = root_dir
#         self.transform = transform

#     def __len__(self):
#         return len(self.labels)

    
#     def _load_image(self, id: int):
#         img_name = os.path.join(self.root_dir, self.labels.iloc[id, 0])
#         return Image.open(img_name)

#     def __getitem__(self, idx):
#         # print(idx)
#         if torch.is_tensor(idx):
#             idx = idx.tolist()

#         img_name = os.path.join(self.root_dir)

#         # image = io.imread(img_name)
#         image = Image.open(img_name).convert('RGB')
#         image = np.array(image)
        
#         # labels = np.array([labels])
        
#         # labels = labels.astype('float').reshape(-1, 2)
#         sample = {'image': image, 'idx': idx}
        
#         if self.transform:
#             # print(sample['labels'].shape)
#             sample = self.transform(sample)
            
#         return image

In [9]:
tfms = transforms.Compose(
    [
        transforms.Resize(IMAGE_SIZE), 
        transforms.ToTensor(),
        transforms.Normalize(
            mean=0.5, 
            std=0.5
        )
   ]
)

In [16]:
image_path = '/content/drive/MyDrive/CS640 Project/Processed Frames'
coco_labels_path = '/content/drive/MyDrive/CS640 Project/COCO_Images_Preprocessed'
# gif_data = GifDataset(csv_file='/content/drive/MyDrive/CS640 Project/Y.csv',
#                                     root_dir= image_path)

In [17]:
transformed_dataset = GifDataset(csv_file='/content/drive/MyDrive/CS640 Project/COCO_labels.csv',
                                           root_dir=coco_labels_path,
                                           transform = tfms
                                          #  transform=transforms.Compose(
                                          #      [ 
                                          #       Rescale(IMAGE_SIZE), 
                                          #       ToTensor(),
                                          #       ]
                                          #  ))
)

In [18]:
train_len = int(TRAIN_PCT * len(transformed_dataset))
train_data, valid_data = random_split(transformed_dataset, [train_len, len(transformed_dataset) - train_len])
train_dl = DataLoader(
    train_data,
    # train_data[0] ,
    BATCH_SIZE, 
    pin_memory=True, 
    shuffle=False, 
    num_workers=1, 
    drop_last=True
)
valid_dl = DataLoader(
    valid_data, 
    # valid_data[0],
    BATCH_SIZE, 
    pin_memory=True, 
    shuffle=False, 
    num_workers=1, 
    drop_last=False
)

# # images, captions = next(iter(train_dl))
# images, idx = next(iter(train_dl))
# images = images
# images.shape, images.min(), images.max(), images.mean(), images.std()

Temp

## Model

In [13]:
# make sure GPT2 appends EOS in begin and end
def build_inputs_with_special_tokens(self, token_ids_0, token_ids_1=None):
    outputs = [self.bos_token_id] + token_ids_0 + [self.eos_token_id]
    return outputs
    
GPT2Tokenizer.build_inputs_with_special_tokens = build_inputs_with_special_tokens
gpt2_tokenizer = GPT2Tokenizer.from_pretrained(DISTIL_GPT2)
# gpt2_tokenizer = AutoTokenizer.from_pretrained('facebook/bart-base')
# set pad_token_id to unk_token_id -> be careful here as unk_token_id == eos_token_id == bos_token_id
gpt2_tokenizer.pad_token = gpt2_tokenizer.unk_token

gpt2_tokenizer_fn = lambda x: gpt2_tokenizer(
    x,
    max_length=MAX_TEXT_LENGTH,
    truncation=True,
    padding=True,
    return_tensors="pt",
)

vit2gpt2 = EncoderDecoderModel.from_encoder_decoder_pretrained(VIT_MODEL, DISTIL_GPT2)

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/762 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/502 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/346M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/353M [00:00<?, ?B/s]

Some weights of GPT2LMHeadModel were not initialized from the model checkpoint at distilgpt2 and are newly initialized: ['transformer.h.5.crossattention.c_proj.bias', 'transformer.h.2.ln_cross_attn.weight', 'transformer.h.5.crossattention.c_attn.weight', 'transformer.h.1.crossattention.q_attn.weight', 'transformer.h.2.crossattention.masked_bias', 'transformer.h.4.crossattention.c_proj.weight', 'transformer.h.0.crossattention.bias', 'transformer.h.0.crossattention.c_proj.bias', 'transformer.h.0.crossattention.c_attn.weight', 'transformer.h.0.crossattention.q_attn.weight', 'transformer.h.1.crossattention.c_proj.bias', 'transformer.h.5.ln_cross_attn.weight', 'transformer.h.4.crossattention.masked_bias', 'transformer.h.3.crossattention.c_proj.bias', 'transformer.h.0.crossattention.c_proj.weight', 'transformer.h.3.crossattention.q_attn.weight', 'transformer.h.3.ln_cross_attn.weight', 'transformer.h.2.crossattention.c_attn.weight', 'transformer.h.5.crossattention.bias', 'transformer.h.1.cros

## Nucleus Sampling
[Paper](https://arxiv.org/pdf/1904.09751.pdf)

In [19]:
def top_k_top_p_filtering(
    next_token_logits: torch.FloatTensor,
    top_k: Optional[float]=None, 
    top_p: Optional[float]=None,
    device: Union[str, torch.device]="cpu",
) -> torch.FloatTensor:
    if top_k is None:
        top_k = next_token_logits.shape[-1]
    if top_p is None:
        top_p = 1.0
        
    p, largest_p_idx = F.softmax(next_token_logits, dim=-1).topk(top_k, dim=-1)
    cumulative_p = p.cumsum(dim=-1)
    threshold_repeated = top_p + torch.zeros((len(p),1)).to(device)
    idx = torch.searchsorted(cumulative_p, threshold_repeated).clip(max=top_k-1).squeeze()
    cutoffs = cumulative_p[torch.arange(len(cumulative_p)), idx]
    censored_p = (cumulative_p <= cutoffs[:, None]) * p
    renormalized_p = censored_p / censored_p.sum(dim=-1, keepdims=True)
    
    final_p = torch.zeros_like(next_token_logits)
    row_idx = torch.arange(len(p)).unsqueeze(1).repeat(1,top_k).to(device)
    final_p[row_idx, largest_p_idx] = renormalized_p.to(final_p.dtype)

    return final_p

def generate_sentence_from_image(model, encoder_outputs, tokenizer, max_text_length: int, device)-> List[str]:
    generated_so_far = torch.LongTensor([[tokenizer.bos_token_id]]*len(encoder_outputs.last_hidden_state)).to(device)
    with torch.no_grad():
        for _ in tqdm(range(max_text_length)):
            attention_mask = torch.ones_like(generated_so_far)
            decoder_out = model(
                decoder_input_ids=generated_so_far, 
                decoder_attention_mask=attention_mask,
                encoder_outputs=encoder_outputs
            )

            next_token_logits = decoder_out["logits"][:, -1, :]
            filtered_p = top_k_top_p_filtering(next_token_logits, top_k=TOP_K, top_p=TOP_P, device=device)
            next_token = torch.multinomial(filtered_p, num_samples=1)
            generated_so_far = torch.cat((generated_so_far, next_token), dim=1)

    return [tokenizer.decode(coded_sentence) for coded_sentence in generated_so_far]

## Training Module (PyTorch Lightning)

In [20]:
descale = transforms.Compose(
    [
        transforms.Normalize(
            mean = [ 0., 0., 0. ],
            std = 1 / 0.5
        ),
        transforms.Normalize(
            mean = -0.5,
            std = [ 1., 1., 1. ]
        ),                           
    ]
)

In [21]:
class LightningModule(pl.LightningModule):
    def __init__(
        self,
        model: nn.Module,
        tokenizer,
        lr: float,
    ):
        super().__init__()
        self.model = model
        self.tokenizer = tokenizer
        self.lr = lr
        
        for name, param in self.model.named_parameters():
            if "crossattention" not in name:
                param.requires_grad = False
        
    def common_step(self, batch: Tuple[torch.FloatTensor, List[str]]) -> torch.FloatTensor:
        images, captions = batch
        tokenized_captions = {
            k: v.to(self.device) for k, v in 
            self.tokenizer(
                captions,
                max_length=MAX_TEXT_LENGTH,
                truncation=True,
                padding=True,
                return_tensors="pt",
            ).items()
        }
        labels = tokenized_captions["input_ids"].clone()
        labels[tokenized_captions["attention_mask"]==0] = LABEL_MASK
        encoder_outputs = self.model.encoder(pixel_values=images)
        outputs = self.model(
            encoder_outputs=encoder_outputs,
            decoder_input_ids=tokenized_captions["input_ids"],
            decoder_attention_mask=tokenized_captions["attention_mask"],
            labels=labels,
            return_dict=True,
        )
        
        return outputs["loss"]
    
    def training_step(self, batch: Tuple[torch.FloatTensor, List[str]], batch_idx: int) -> torch.FloatTensor:
        loss = self.common_step(batch)
        self.log(name="Training loss", value=loss, on_step=True, on_epoch=True)
        
        return loss
        
    def validation_step(self, batch: Tuple[torch.FloatTensor, List[str]], batch_idx: int):
        loss = self.common_step(batch)
        self.log(name="Validation loss", value=loss, on_step=True, on_epoch=True)

        images, actual_sentences = batch
        
        if batch_idx == 0:
            encoder_outputs = self.model.encoder(pixel_values=images.to(self.device))
            generated_sentences = generate_sentence_from_image(
                self.model, 
                encoder_outputs, 
                self.tokenizer, 
                MAX_TEXT_LENGTH,
                self.device
            )
            images = [wandb.Image(transforms.ToPILImage()(descale(image))) for image in images]
            # images = [wandb.Image(transforms.ToPILImage()) for image in images]

            data = list(map(list, zip(images, actual_sentences, generated_sentences)))
            columns = ["Images", "Actual Sentence", "Generated Sentence"]
            table = wandb.Table(data=data, columns=columns)
            self.logger.experiment.log({f"epoch {self.current_epoch} results": table})
                        
    def on_after_backward(self):
        if self.trainer.global_step % 50 == 0:  # don't make the tf file huge
            for name, param in self.model.named_parameters():
                if "weight" in name and not "norm" in name and param.requires_grad:
                    self.logger.experiment.log(
                        {f"{name}_grad": wandb.Histogram(param.grad.detach().cpu())}
                    )
                    self.logger.experiment.log(
                        {f"{name}": wandb.Histogram(param.detach().cpu())}
                    )
    
    # def forward(self, images):
    #      encoder_outputs = self.model.encoder(pixel_values=images.to(self.device))
    #      generated_sentences = generate_sentence_from_image(
    #             self.model, 
    #             encoder_outputs, 
    #             self.tokenizer, 
    #             MAX_TEXT_LENGTH,
    #             self.device
    #         )
         
    #      return generated_sentences

    def configure_optimizers(self) -> torch.optim.Optimizer:
        return torch.optim.Adam(self.model.parameters(), lr=self.lr)
            

In [22]:
checkpoint_callback = ModelCheckpoint(
  dirpath="checkpoints",
  filename="best-checkpoint",
  save_top_k=1,
  verbose=True,
  monitor="Validation loss",
  mode="min"
)

In [23]:
!mkdir -p /content/sample_data/logs
lightning_module = LightningModule(
    vit2gpt2,
    gpt2_tokenizer,
    LR
)

trainer = pl.Trainer(
    max_epochs=EPOCHS,
    gpus=torch.cuda.device_count(),
    gradient_clip_val=1.0,
    logger=WandbLogger("time kam h", "/content/sample_data/logs", project="Vit2GPT2"),
    precision=16,
    num_sanity_val_steps=0,
    callbacks = [checkpoint_callback],
    # logger = TensorBoardLogger('xas', name = 'aakash')
)
trainer.fit(lightning_module, train_dl, valid_dl)

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: aakash0017 (640-project). Use `wandb login --relogin` to force relogin


INFO:pytorch_lightning.utilities.rank_zero:Using 16bit native Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                | Params
----------------------------------------------
0 | model | EncoderDecoderModel | 182 M 
----------------------------------------------
14.2 M    Trainable params
168 M     Non-trainable params
182 M     Total params
364.970   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

  0%|          | 0/32 [00:00<?, ?it/s]

...

  0%|          | 0/32 [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 8, global step 10683: 'Validation loss' was not in top 1


Validation: 0it [00:00, ?it/s]

  0%|          | 0/32 [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 9, global step 11870: 'Validation loss' was not in top 1
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.


In [ ]:
lightning_modul = LightningModule.load_from_checkpoint(
    checkpoint_path = '/content/checkpoints/best-checkpoint-v1.ckpt',
    model = vit2gpt2,
    tokenizer = gpt2_tokenizer,
    lr = LR
)

In [ ]:
input.shape

torch.Size([244, 200])

In [ ]:
#Creating list of all the frames
from PIL import Image, ImageSequence
path = '/content/sample_data/1.gif'
path_to_save = ''
frame_list = []
im = Image.open(path).convert('RGB')
for frame in ImageSequence.Iterator(im):
  f = frame.copy()
  frame_list.append(f)

In [ ]:
input = {'image': np.array(frame_list[0]), 'labels': 'hello'}

In [ ]:
image_path = '/content/sample_data/gif/1.gif'
test_set = TestLoader(
                      root_dir=image_path,
                      )

In [ ]:
transform_=transforms.Compose([
    Rescale(IMAGE_SIZE),  
    ToTensor()
    ])

In [ ]:
i = transform_(input)[0]

In [ ]:
i = i.unsqueeze(0)

In [ ]:
pred = lightning_modul.model.encoder(pixel_values=i)


In [ ]:
generated_sentences = generate_sentence_from_image(
    lightning_module.model, 
    pred, 
    gpt2_tokenizer, 
    MAX_TEXT_LENGTH,
    device = 'cpu'
)

  0%|          | 0/32 [00:00<?, ?it/s]

In [ ]:
pred

In [ ]:
generated_sentences

['<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|>']